In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder \
.master("local[*]") \
.appName('test') \
.getOrCreate()

In [ ]:
df_green = spark.read.parquet('data/raw/green')

In [ ]:
df_green  = df_green \
  .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
  .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
df_green.printSchema()

In [ ]:
df_yellow = spark.read.parquet('data/raw/yellow')

In [ ]:
df_yellow  = df_yellow \
  .withColumnRenamed('tpep_pickup_datetime','pickup_datetime') \
  .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
df_yellow.printSchema()

In [ ]:
common_columns = []

yellow_columns = set(df_yellow.columns)
for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [ ]:
common_columns

In [ ]:
set(df_green.columns) & set(df_yellow.columns)

In [ ]:
df_green_sel = df_green.select(common_columns).withColumn('service_type',F.lit('green'))

In [ ]:
df_yellow_sel = df_yellow.select(common_columns).withColumn('service_type',F.lit('yellow'))

In [ ]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [ ]:
df_trips_data.groupBy('service_type').count().show()

In [ ]:
df_trips_data.createOrReplaceTempView('trips_data')

In [ ]:
spark.sql (
    """
    SELECT * FROM trips_data LIMIT 10;
    """
).show()

In [ ]:
spark.sql(
    """
    SELECT 
        service_type,
        COUNT(1) AS count
    FROM 
        trips_data
    GROUP BY
        service_type
    """
).show()


In [ ]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [ ]:
df_result.show()

In [ ]:
df_result.coalesce(1).write.parquet('data/raw/report/revenue/', mode='overwrite')